In [ ]:
!nvidia-smi

Sun Apr  4 20:16:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastcore onnx onnxruntime sentencepiece seqeval rouge-score
    !pip install -Uqq --no-deps fastai ohmeow-blurr
    !pip install -Uqq transformers datasets wandb 

In [ ]:
from fastai.text.all import *
from fastai.callback.wandb import *

In [ ]:
def read_text(fn):
    return open(fn).read()

In [ ]:
path = untar_data(URLs.IMDB)

## Setup

In [ ]:
model_name = 'distilbert-base-uncased'

max_len = 512
bs = 8
val_bs = 16

## Tracking

In [ ]:
# !wandb login

In [ ]:
import wandb

WANDB_NAME = f'imdb-{model_name}-simple'
GROUP = f'IMDB-{model_name}-simple'
NOTES = f'Simple finetuning {model_name} with RAdam lr=2e-5'
CONFIG = {}
TAGS =[model_name,'imdb','radam','simple']

In [ ]:
wandb.init(reinit=True, project="vat", entity="fastai_community",
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

wandb: Currently logged in as: fastai_community (use `wandb login --relogin` to force relogin)


## Training

In [ ]:
def _to_device(e, device):
    if hasattr(e, 'to'): return e.to(device)
    elif isinstance(e, dict):
        for _, v in e.items():
            if hasattr(v, 'to'): v.to(device)
        return {k:(v.to(device) if hasattr(v, 'to') else v) for k, v in e.items()}

In [ ]:
@patch
def one_batch(self:Learner, i, b):
        self.iter = i
        b_on_device = tuple(_to_device(e, self.dls.device) for e in b) if self.dls.device is not None else b
        self._split(b_on_device)
        self._with_events(self._do_one_batch, 'batch', CancelBatchException)

In [ ]:
from transformers import *

from blurr.data.all import *
from blurr.modeling.all import *

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
task = HF_TASKS_AUTO.SequenceClassification
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(model_name, task=task, tokenizer_kwargs={'max_len':512})

In [ ]:
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), CategoryBlock)
dblock = DataBlock(blocks=blocks, 
                   get_items=get_text_files,
                   get_x = read_text,
                   get_y=parent_label,
                   splitter=GrandparentSplitter(valid_name='test'))

dls = dblock.dataloaders(path, bs=bs, val_bs=val_bs, shuffle=True)

In [ ]:
model = HF_BaseModelWrapper(hf_model)
learn = Learner(dls,
                model,
                opt_func=RAdam,
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.create_opt() 
learn.unfreeze()
learn.blurr_summary()

HF_BaseModelWrapper (Input shape: 8 x 512)
Layer (type)         Output Shape         Param #    Trainable 
                     8 x 512 x 768       
Embedding                                 23440896   True      
Embedding                                 393216     True      
LayerNorm                                 1536       True      
Dropout                                                        
Dropout                                                        
Linear                                    590592     True      
Linear                                    590592     True      
Linear                                    590592     True      
Linear                                    590592     True      
LayerNorm                                 1536       True      
Dropout                                                        
____________________________________________________________________________
                     8 x 512 x 3072      
Linear                      

In [ ]:
learn.show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, MixedPrecision, Recorder, ProgressCallback]
  Start Epoch Loop
     - before_epoch   : [Recorder, ProgressCallback]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - before_batch   : [HF_BaseModelCallback, MixedPrecision]
         - after_pred     : [HF_BaseModelCallback, MixedPrecision]
         - after_loss     : [HF_BaseModelCallback, MixedPrecision]
         - before_backward: [MixedPrecision]
         - before_step    : [MixedPrecision]
         - after_step     : [MixedPrecision]
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder, ProgressCallback]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder, ProgressCallback]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - **CBs same as train bat

### simple finetuning

In [ ]:
learn.fit_one_cycle(4, 2e-5, cbs=WandbCallback(log_preds=False, log_model=False))

Could not gather input dimensions


epoch,train_loss,valid_loss,accuracy,time
0,0.243032,0.246754,0.890720,11:27
1,0.146905,0.185891,0.926920,11:30
2,0.077715,0.223364,0.934840,11:32
3,0.034727,0.265831,0.934840,11:32


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
Exception ignored in: <finalize object at 0x7f8f04744700; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 936, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.py", line 483, in rmtree
    orig_st = os.lstat(path)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpq